In [2]:
import numpy as np
import os
import pandas as pd
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
# For charts 
from sklearn import svm 
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline 
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')

# Libraries for analysis 
import pandas as pd
import xlrd 
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold


from numpy.random import choice
import math 
import numpy as np

    
data_path = os.path.join('.','tests','test_output')

X = np.load(os.path.join(data_path,'x_split_alt_0.npy'),mmap_mode='r')
Y = np.load(os.path.join(data_path,'y_split_alt_0.npy'),mmap_mode='r')


In [ ]:

 # The following function take as inputs the weights 
#data(for each symbol), the samples 
# to test ,and their labels   
def adaboostM1(XX,yy,weights_data):
    # List of classifiers 
    list_clf =[]
    # List of classifiers constants 
    list_clfwei = [] 
    
    T = 50 # maximum number of classifiers
    t =0 
    while t < T:
        Length_data = range(len(XX)) 
        
        # Selecting a specific number of samples
        Samples = choice(Length_data,100,True,weights_data) 
        examples =XX[Samples]
        labels = yy[Samples]

        # Train classifier on those examples
        clfAda = (svm.SVC(kernel='linear', C=0.1))
        clfAda.fit(examples,labels)

        # Get error 
        error = 1 - clfAda.score(XX,yy,weights_data)
        
        # Establish condition to skip rest of the loop 
        if (error>0.5):
            continue
    
        # Find Beta
        B_t = error/(1-error); 

        # Predict label for the whole data set
        guesses = clfAda.predict(XX)
        
        # Compare predictions to actual labels
        for i in range(len(guesses)):
            if yy[i] == guesses[i]:
                # Penalize the weights of the data when its right
                weights_data[i] = B_t*weights_data[i]; 

        # Normalize the data 
        sumofweights = sum(weights_data)
        weights_data = [x/sumofweights for x in weights_data]   

        # We have 50 classifiers in total, each one has a constant
        list_clf.append(clfAda)
        list_clfwei.append(math.log(1/B_t,2))
        t= t+1
        
    return list_clf, list_clfwei
 # Let's have 50 lists that we can multiply to get one final lists 
from sklearn.model_selection import RepeatedKFold
def compute(XX,yy,Xt,yt): 
    # Runs the adaboost, takes as inputs the values the inputs the 
    #training data and the 
    #validation data as well 
    # Re-initialize the weights 
    weights_data = [1/len(XX)]*len(XX)
    # Now compute the accuracy for the ensemble classifier     
    list_clf, list_clfwei =  adaboostM1(XX,yy,weights_data)
    
    # Guess all data and add it
    guesses2 = 0 
    

    d = np.zeros((len(Xt),len(np.unique(yt))))

    for i in range(len(list_clf)):       
        l = list_clf[i].predict(Xt) 
        r = np.asmatrix(l)
        
        for j in range(r.size):
            d[j,r[0,j]] += list_clfwei[i]
        
    guesses2 = np.argmax(d, axis=1)


    # Compute the signs of all outputs 
    acc = 0 
    
    # Check how the guesses compare to the actual number
    for i in range(len(guesses2)):
        if yt[i] == guesses2[i]:
            acc = acc+1  
    acc = acc/len(guesses2)
    return acc

listacc = [] 

# Do the 10 times 10 fold cross-validation using the "computing" function
rkf2 = RepeatedKFold(n_splits=2, n_repeats=1, random_state=None)
for train_index, test_index in rkf2.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    listacc.append(compute(X_train,y_train,X_test,y_test))
    
# Finalize and print results
listacc = [x*100 for x in listacc]
print("The variance as a percentage is", np.var(listacc))
print("The mean as a percentage is", np.mean(listacc))